In [ ]:
from lusidtools.jupyter_tools import toggle_code

"""Example Data Load

Loads some example data into LUSID

Attributes
----------
example data
"""

toggle_code("Toggle Docstring")

# Example Data Load

In [ ]:
## Importing required packages
import os
import pandas as pd
import datetime
from datetime import timezone
import pytz

import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [ ]:
# construct the API we want to use
CutLabelDefinitionsApi = api_factory.build(lusid.api.CutLabelDefinitionsApi)
PortfoliosApi = api_factory.build(lusid.api.PortfoliosApi)
TransactionPortfoliosApi = api_factory.build(lusid.api.TransactionPortfoliosApi)
QuotesApi = api_factory.build(lusid.api.QuotesApi)
ConfigurationRecipeApi = api_factory.build(lusid.api.ConfigurationRecipeApi)
PortfolioGroupsApi = api_factory.build(lusid.api.PortfolioGroupsApi)
InstrumentsApi = api_factory.build(lusid.api.InstrumentsApi)
ValuationsApi = api_factory.build(lusid.api.AggregationApi)

In [ ]:
scope = "Finbourne-Examples"
market_data_scope = "FinbourneMarketData"

datafile = "ExampleData.xlsx"
date = datetime.datetime(2021, 1, 1, 0, 0, 0, 0, tzinfo=timezone.utc)

In [ ]:
## Loading Data
portfolios = pd.read_excel(datafile,sheet_name = 'Portfolios')
holdings = pd.read_excel(datafile,sheet_name = 'Holdings')
transactions = pd.read_excel(datafile,sheet_name = 'Transactions')
instruments = pd.read_excel(datafile,sheet_name = 'Instruments')
quotes = pd.read_excel(datafile,sheet_name = 'Quotes').fillna('')
rates = pd.read_excel(datafile,sheet_name = 'Rates').fillna('')
recipes = pd.read_excel(datafile,sheet_name = 'Recipes')
groups = pd.read_excel(datafile,sheet_name = 'Groups')

### Creating CutLabels

In [ ]:
cutLabels = [["LDN_Open","LondonOpen","London Market Open Time", models.CutLocalTime(8, 0), "Europe/London"],
             ["LDN_Close","LondonClose","London Market Close Time", models.CutLocalTime(16, 30), "Europe/London"],
             ["NY_Close","NewYorkClose","New York Market Close Time", models.CutLocalTime(16, 00), "America/New_York"],
             ["NY_Open","NewYorkOpen","New York Market Open Time", models.CutLocalTime(9, 30), "America/New_York"]]

createdCutLabels = []

def CreateCutLabel(cutLabel, overwrite=True):
    '''
    Input: 
        cutlabel (list): ordered list where 
                            [0] == code (str), 
                            [1] == display_name (str), 
                            [2] == description (str), 
                            [3] == cut_local_time (models.cutLabelTime),
                            [4] == time_zone (str)
        overwrite (bool): whether to overwrite existing cutlabel in the provided code if it exists, defaults to True
    Output:
        resp (lusid.models.cut_label_definition.CutLabelDefinition): Cut Label definition of the created cut label
    '''
    ## Checks if cutLabel already exist
    try:
        # Check if cutlabel with code already exist
        resp = CutLabelDefinitionsApi.get_cut_label_definition(code=cutLabel[0])
        if overwrite:
            resp = CutLabelDefinitionsApi.delete_cut_label_definition(code=cutLabel[0])
        else:
            return resp

    except:
        pass

    cutLabelReq = models.CreateCutLabelDefinitionRequest(code=cutLabel[0],
                                                         display_name=cutLabel[1],
                                                         description=cutLabel[2],
                                                         cut_local_time=cutLabel[3],
                                                         time_zone=cutLabel[4])
    
    CutLabelDefinitionsApi.create_cut_label_definition(create_cut_label_definition_request=cutLabelReq)

    return CutLabelDefinitionsApi.get_cut_label_definition(code=cutLabel[0])
    
    
for c in cutLabels:

    response = CreateCutLabel(c)
    createdCutLabels.append(response)

createdCutLabels

### Creating Instruments

In [ ]:
createdInstruments = []

def CreateInstrument(instrument, overwrite=True):
    '''
    Input: 
        instrument (dict): dictionary with fields 
                            'name' == name of the instrument (str), 
                            'Figi' == Figi identifier (str), 
                            'P:Instrument/default/Currency' == default currency (str)
        overwrite (bool): whether to overwrite existing instrument (hard-delete) in the provided code if it exists, defaults to True
    Output:
        resp (lusid.models.instrument.Instrument): Instrument that was created
    '''
    ## Checks if Instrument already exist
    try:
        # Check if Instrument with FIGI already exist
        resp = InstrumentsApi.get_instrument("Figi",instrument['Figi'])
        if overwrite:
            resp = InstrumentsApi.delete_instruments(request_body = [resp.lusid_instrument_id],delete_mode = 'Hard')
        else:
            return resp

    except:
        pass

    instrumentReq = {"inst":models.InstrumentDefinition(
                                    name=instrument["name"],
                                    identifiers={"Figi":models.InstrumentIdValue(value = instrument['Figi'])},
                                    properties=[lusid.ModelProperty(key="Instrument/default/Currency",
                                                                    value=lusid.PropertyValue(label_value=instrument['P:Instrument/default/Currency'])
                                                                   )
                                               ]
                                    )
                    }
    InstrumentsApi.upsert_instruments(request_body=instrumentReq)
    return InstrumentsApi.get_instrument("Figi",instrument['Figi'],property_keys=['Instrument/default/Currency'])

for i,inst in instruments.iterrows():
    response = CreateInstrument(inst.to_dict())
    createdInstruments.append(response)

createdInstruments

### Creating Portfolios

In [ ]:
createdPortfolios = []

def CreatePortfolio(portfolio, overwrite=True):
    '''
    Input: 
        portfolio (dict): dictionary with fields 
                            'Code' == Code for the portfolio (str), 
                            'Display Name' == Display name of the identifier (str), 
                            'Base Currency' == Base Currency (str)
        overwrite (bool): whether to overwrite existing portfolio (hard-delete) in the provided code if it exists, defaults to True
    Output:
        resp (lusid.models.portfolio.Portfolio): Portfolio that was created
    '''
    ## Checks if Portfolio already exist
    try:
        # Check if Portfolio with code already exist
        resp = PortfoliosApi.get_portfolio(scope=scope,code=portfolio['Code'])
        if overwrite:
            resp = PortfoliosApi.delete_portfolio(scope=scope,code=portfolio['Code'])
        else:
            return resp

    except:
        pass

    portReq = models.CreateTransactionPortfolioRequest(code=portfolio['Code'],
                                                       display_name=portfolio['Display Name'],
                                                       base_currency=portfolio['Base Currency'],
                                                       created=date)
    resp = TransactionPortfoliosApi.create_portfolio(scope=scope,create_transaction_portfolio_request=portReq)
    return PortfoliosApi.get_portfolio(scope=scope,code=portfolio['Code'])

for i,port in portfolios.iterrows():
    response = CreatePortfolio(port.to_dict())
    createdPortfolios.append(response)

createdPortfolios

### Creating Portfolio Groups

In [ ]:
portGroups = groups[['Group Name', 'Display Name', 'Description','Created Date']].drop_duplicates().reset_index(drop=True)
portGroups['Sub-groups'] = [list(set(groups.query(f"`Group Name`=='{grp['Group Name']}' and `Display Name`=='{grp['Display Name']}' and Description == '{grp['Description']}' and `Created Date` == '{grp['Created Date']}' ").dropna(subset=['Sub-groups'])['Sub-groups'])) 
                            for i,grp in portGroups.iterrows()]
portGroups['Portfolios'] = [list(set(groups.query(f"`Group Name`=='{grp['Group Name']}' and `Display Name`=='{grp['Display Name']}' and Description == '{grp['Description']}' and `Created Date` == '{grp['Created Date']}' ").dropna(subset=['Portfolios'])['Portfolios'])) 
                            for i,grp in portGroups.iterrows()]
portfolioGroups = [portGroups.iloc[i].to_dict() for i in range(len(portGroups))]
portfolioGroups.reverse()


createdPortfolioGroups = []

def CreatePortfolioGroup(portfolioGroup, overwrite=True):
    '''
    Input: 
        portfolioGroup (dict): dictionary with fields 
                                'Group Name': code of the portfolio group (str),
                                'Display Name': Display name of the portfolio group (str),
                                'Description': Description of the portfolio group (str),
                                'Created Date': Date of creation in YYYY-MM-DDTHH:mm:ssZ (str),
                                'Sub-groups': list of codes of sub portfolio groups that must exist (list[(str)]),
                                'Portfolios': list of codes of portfolio that must exist (list[(str)])
        overwrite (bool): whether to overwrite existing portfolioGroup (hard-delete) in the provided code if it exists, defaults to True
    Output:
        resp (lusid.models.portfolio_group.PortfolioGroup): PortfolioGroup that was created
    '''
    ## Checks if PortfolioGroup already exist
    try:
        # Check if PortfolioGroup with code already exist
        resp = PortfolioGroupsApi.get_portfolio_group(scope=scope, code= portfolioGroup['Group Name'])
        if overwrite:
            resp = PortfolioGroupsApi.delete_portfolio_group(scope=scope, code= portfolioGroup['Group Name'])
        else:
            return resp

    except:
        pass

    portGrpReq = models.CreatePortfolioGroupRequest(code=portfolioGroup['Group Name'],
                                                    created=portfolioGroup['Created Date'],
                                                    values=[lusid.ResourceId(scope=scope, code = port) 
                                                            for port in portfolioGroup['Portfolios']], 
                                                    sub_groups=[lusid.ResourceId(scope=scope, code = subGrp) 
                                                                for subGrp in portfolioGroup['Sub-groups']],
                                                    description=portfolioGroup['Description'],
                                                    display_name=portfolioGroup['Display Name'])

    PortfolioGroupsApi.create_portfolio_group(scope=scope,create_portfolio_group_request=portGrpReq)
    return PortfolioGroupsApi.get_portfolio_group(scope=scope, code= portfolioGroup['Group Name'])

for grp in portfolioGroups:
    response = CreatePortfolioGroup(grp)
    createdPortfolioGroups.append(response)

createdPortfolioGroups

### Set Holdings

In [ ]:
setHoldingsResponse = []

def setHoldings(scope, port, port_holdings):
    '''
    Input: 
        port_holdings (pandas.DataFrame): pandas dataframe with the following columns 
                                'Portfolio Code': code of the portfolio (str),
                                'instrument_uid': identifier of the instrument (identifierkey:value) (dict),
                                'Instrument Name': name of the instrument (str),
                                'Holding Type': type of the holding (str),
                                'units': number of the given instrument (float),
                                'price': price of the instrument (float),
                                'cost.currency': CCY code (str),
                                'purchase_date': purchase date in UTC (pandas._libs.tslibs.timestamps.Timestamp),
                                'settlement_date': settlmeent date in UTC (pandas._libs.tslibs.timestamps.Timestamp),
                                'cost.amount': cost of the transaction to get this holding in cost currency (float),
                                'portfolio_cost': cost of the transaction to get this holding in portfolio currency (float)
    Output:
        resp (lusid.models.adjust_holding.AdjustHolding): AdjustHolding response
    '''
    holdingsReq = [models.AdjustHoldingRequest(instrument_identifiers={hold['instrument_uid'].split(':')[0]:hold['instrument_uid'].split(':')[1]},
                                           tax_lots=[models.TargetTaxLotRequest(units=hold['units'],
                                                                                cost=models.CurrencyAndAmount(amount=hold['cost.amount'],
                                                                                                              currency=hold['cost.currency']),
                                                                                portfolio_cost=hold['portfolio_cost'],
                                                                                price=hold['price'],
                                                                                purchase_date=pytz.timezone('UTC').localize(hold['purchase_date'].to_pydatetime()),
                                                                                settlement_date=pytz.timezone('UTC').localize(hold['settlement_date'].to_pydatetime()))],
                                          currency=hold['cost.currency']) 
               for i,hold in port_holdings.iterrows()]


    resp = TransactionPortfoliosApi.set_holdings(scope=scope,
                                          code=port,
                                          effective_at=date,
                                          adjust_holding_request=holdingsReq)
    return resp

ports = holdings['Portfolio Code'].unique()
for port in ports:
    port_holdings = holdings.query(f"`Portfolio Code` == '{port}'")
    response = setHoldings(scope, port, port_holdings)
    setHoldingsResponse.append(response)

setHoldingsResponse

### Create Transactions

In [ ]:
upsertTransactionResponse = []

def upsertTransactions(scope, port, port_trans):
    '''
    Input: 
        port_trans (pandas.DataFrame): pandas dataframe with the following columns 
                                'Portfolio Code': code of the portfolio (str),
                                'transaction_id' : ID of this transaction (str),
                                'instrument_uid': identifier of the instrument (identifierkey:value) (dict),
                                'type' : type of this transaction (str),
                                'transaction_date' : transaction date in UTC (pandas._libs.tslibs.timestamps.Timestamp),
                                'settlement_date': settlement date in UTC (pandas._libs.tslibs.timestamps.Timestamp),
                                'units': number of the given instrument (float),
                                'total_consideration.currency' : CCY code (str),
                                'transaction_price.price' : price at which the instrument was transacted at (float),
                                'total_consideration.amount' : amount of the particular instrument transacted (float),
                                'source : source of this transaction (str)'
    Output:
        resp (lusid.models.adjust_holding.AdjustHolding): AdjustHolding response
    '''
    transReq = [models.TransactionRequest(transaction_id=trans['transaction_id'], 
                                          type=trans['type'],
                                          instrument_identifiers={trans['instrument_uid'].split(':')[0]:trans['instrument_uid'].split(':')[1]}, 
                                          transaction_date=pytz.timezone('UTC').localize(trans['transaction_date'].to_pydatetime()),
                                          settlement_date=pytz.timezone('UTC').localize(trans['settlement_date'].to_pydatetime()), 
                                          units=trans['units'],
                                          transaction_price=models.TransactionPrice(price=trans['transaction_price.price']),
                                          total_consideration=models.CurrencyAndAmount(amount=trans['total_consideration.amount'],
                                                                                       currency=trans['total_consideration.currency']),
                                          source=trans['source'])
                for i,trans in port_trans.iterrows()]

    resp = TransactionPortfoliosApi.upsert_transactions(scope=scope,
                                                        code=port,
                                                        transaction_request=transReq)
    return resp

ports = transactions['Portfolio Code'].unique()
for port in ports:
    port_trans = transactions.query(f"`Portfolio Code` == '{port}'")
    response = upsertTransactions(scope, port, port_trans)
    upsertTransactionResponse.append(response)

upsertTransactionResponse

### Create Quotes

In [ ]:
quotesReq = {}

for i,quote in quotes.iterrows():
    quoteModel = lusid.UpsertQuoteRequest(quote_id=lusid.QuoteId(quote_series_id = lusid.QuoteSeriesId(provider = quote['provider'], 
                                                                                                       price_source=quote["source"],
                                                                                                       instrument_id = quote['instrument_uid'], 
                                                                                                       instrument_id_type=quote['instrument_uid_type'], 
                                                                                                       quote_type=quote['quote_type'], 
                                                                                                       field=quote['field']),
                                                                 effective_at= pytz.timezone('UTC').localize(quote['effective_at'].to_pydatetime())), 
                                          metric_value=lusid.MetricValue(value=quote['metric_value'],
                                                                         unit=quote['metric_unit']))
    quotesReq[f"{quote['instrument_uid']}-{str(quote['effective_at'])}-{quote['field']}"] = quoteModel
    
## Upserting Quotes
quoteNumbers = len(quotesReq)
maxPerupsert = 500
numberOfReq = quoteNumbers/maxPerupsert
quoteKeys = list(quotesReq.keys())
upsertResults = []
for i in range(int(numberOfReq)):
    requestKeys = quoteKeys[i*500 : (i+1)*500]
    request = dict(zip(requestKeys, [quotesReq[key] for key in requestKeys]))
    upsertResults.append(QuotesApi.upsert_quotes(scope=market_data_scope, request_body=request))
    
if quoteNumbers - ((i+1)*500) > 0:
    requestKeys = quoteKeys[((i+1)*500) : ]
    request = dict(zip(requestKeys, [quotesReq[key] for key in requestKeys]))
    upsertResults.append(QuotesApi.upsert_quotes(scope=market_data_scope, request_body=request))

    
print("Failures: ", [i.failed for i in upsertResults])

### Create FX Rates

In [ ]:
ratesReq = {}

for i,rate in rates.iterrows():
    quoteModel = lusid.UpsertQuoteRequest(quote_id=lusid.QuoteId(quote_series_id = lusid.QuoteSeriesId(provider = 'Client', 
                                                                                                       price_source=rate["source"],
                                                                                                       instrument_id = rate['instrument_uid'], 
                                                                                                       instrument_id_type=rate['instrument_uid_type'], 
                                                                                                       quote_type=rate['quote_type'], 
                                                                                                       field=rate['field']),
                                                                 effective_at= pytz.timezone('UTC').localize(rate['effective_at'].to_pydatetime())), 
                                          metric_value=lusid.MetricValue(value=rate['metric_value'],
                                                                         unit=rate['metric_unit']),
                                          lineage= "testckg")
    ratesReq[f"{rate['instrument_uid']}-{str(rate['effective_at'])}-{rate['field']}"] = quoteModel
    
## Upserting Quotes
quoteNumbers = len(ratesReq)
maxPerupsert = 500
numberOfReq = quoteNumbers/maxPerupsert
quoteKeys = list(ratesReq.keys())
upsertResults = []
for i in range(int(numberOfReq)):
    requestKeys = quoteKeys[i*500 : (i+1)*500]
    request = dict(zip(requestKeys, [ratesReq[key] for key in requestKeys]))
    upsertResults.append(QuotesApi.upsert_quotes(scope=market_data_scope, request_body=request))
    
if quoteNumbers - ((i+1)*500) > 0:
    requestKeys = quoteKeys[((i+1)*500) : ]
    request = dict(zip(requestKeys, [ratesReq[key] for key in requestKeys]))
    upsertResults.append(QuotesApi.upsert_quotes(scope=market_data_scope, request_body=request))

    
print("Failures: ", [i.failed for i in upsertResults])

### Create Recipes

In [ ]:
createdRecipes = []

def createRecipe(code, rec_rules):
    '''
    Input: 
        rec_rules (pandas.DataFrame): pandas dataframe with the following columns 
                                'Description' : description of the recipe (str),
                                'Key' : market data key (str),
                                'Supplier' : supplier of the market data (str),
                                'QuoteType' : type of the quote (str),
                                'QuoteInterval' : interval of the quote (str),
                                'Field' : field of the quote (str)
    Output:
        resp (lusid.models.adjust_holding.AdjustHolding): AdjustHolding response
    '''
    description = rec_rules.Description.unique()[0]
    supplier = "Client" #rec_rules.Supplier.unique()[0]
    recipeReq = models.UpsertRecipeRequest(
                        configuration_recipe=lusid.ConfigurationRecipe(
                            scope=scope,
                            code=code,
                            description=description,
                            market=models.MarketContext(market_rules=[
                                models.MarketDataKeyRule(key=rule['Key'],
                                   supplier=rule['Supplier'],
                                   data_scope=market_data_scope,
                                   quote_type=rule['QuoteType'],
                                   field=rule['Field'],
                                   quote_interval=rule['QuoteInterval'])
                            for i,rule in rec_rules.iterrows()],
                            suppliers=lusid.MarketContextSuppliers(
                                commodity=supplier,
                                credit=supplier,
                                equity=supplier,
                                fx=supplier,
                                rates=supplier),
                           options=models.MarketOptions(
                                default_supplier='DataScope',
                                default_instrument_code_type='Figi',
                                default_scope=market_data_scope)
                               )
                        )
                ) 

    ConfigurationRecipeApi.upsert_configuration_recipe(upsert_recipe_request = recipeReq)
    return ConfigurationRecipeApi.get_configuration_recipe(scope=scope, code=code)

recs = recipes['Code'].unique()
for rec in recs:
    rec_rules = recipes.query(f"Code == '{rec}'")
    response = createRecipe(rec, rec_rules)
    createdRecipes.append(response)

createdRecipes

In [ ]:
[i.value.code for i in createdRecipes]

### Test Aggregation using created Recipe

In [ ]:
AGG_INSTR = "Instrument/default/Name"
AGG_UID = "Instrument/default/LusidInstrumentId"
AGG_PV = "Holding/default/PV"
AGG_PRC = "Holding/default/Price"
AGG_UNITS = "Holding/default/Units"
AGG_COST = "Holding/default/Cost"
AGG_TYPE = "Holding/default/Type"
AGG_RATE = "Holding/default/ExchangeRate"


metrics = [
    models.AggregateSpec(AGG_INSTR, "Value"),
    models.AggregateSpec(AGG_UID, "Value"),
    models.AggregateSpec(AGG_TYPE, "Value"),
    models.AggregateSpec(AGG_PV, "Value"),
    models.AggregateSpec(AGG_UNITS, "Value"),
    models.AggregateSpec(AGG_COST, "Value"),
    models.AggregateSpec(AGG_RATE, "Value"),
    models.AggregateSpec(AGG_PRC, "Value"),
    ]


valuationRequest = models.ValuationRequest(
    recipe_id=models.ResourceId(scope, createdRecipes[0].value.code),
    valuation_schedule=models.ValuationSchedule(effective_at=pd.to_datetime('2021-05-17', utc=True)),
    portfolio_entity_ids=[
        models.PortfolioEntityId(
                scope=createdPortfolios[1].id.scope,
                code=createdPortfolios[1].id.code,
                portfolio_entity_type="SinglePortfolio")
    ],
    metrics=metrics
)

ValuationsApi.get_valuation(valuation_request=valuationRequest)